# CURRENT CLICK 

**Setup code**

Imports the Mercury overlay from the Red Pitaya library and gives it an alias "FPGA". Creates an object called "overlay" which represents the Mercury overlay on the Red Pitaya board.

In [ ]:
from redpitaya.overlay.mercury import mercury as FPGA
import time

# Initialize the FPGA overlay
overlay = FPGA()

LED = FPGA.led

mikrobus1AinPin = FPGA.analog_in(0)
mikrobus2AinPin = FPGA.analog_in(1)

def convert_to_engineering_unit(number, units):
    magnitude = number
    unit_index = 0

    while magnitude <= 1.0 and unit_index < len(units) - 1:
        magnitude *= 1000.0
        unit_index += 1

    print("Value: {:.4f} {} \n".format(magnitude, units[unit_index]))

def get_current(pin, resistor_value):
    # Get ADC value from the specified pin
    value = pin.read()
    max_current = 3.3 / (20.0 * resistor_value)
    print("Minimum current is 2mA, maximum is:")
    convert_to_engineering_unit(max_current, ["A", "mA", "μA", "nA"])

    # Calculate current
    voltage = value / 20.0
    # gain of circuit is 20
    current = voltage / resistor_value

    return current

def recommend_resistor(current):
    if current != 0:
        resistor = 0.075 / current
        print("The recommended resistor value for best measurement (R) is {:.4f} ohms.\n".format(resistor))
        return resistor
    else:
        print("Error: Current (I) cannot be zero.\n")
        return -1
        # Return -1 to indicate an error

The example shows how to read the current through a shunt resistor. The program, however, does not know how many ohms your shunt resistor has. That is why the shunt resistor value must be input when prompted by the program. Also, the shunt resistor should be accurate to at least three decimals (for example, with a multimeter). That way, the current measurement will be most accurate.

Refrain from going above or below the minimum and maximum current measurement.

For best measurements, please use the recommended resistor.

In [ ]:
while True:
    print("Enter shunt resistor value in Ohms: ")
    # circuit 5 ohms
    resistor_value = float(input())
    current = get_current(mikrobus1AinPin, resistor_value)

    convert_to_engineering_unit(current, ["A", "mA", "μA", "nA"])
    recommend_resistor(current)
    time.sleep(1)